In [1]:
import pandas as pd
import random
random.seed(2)

which_wiki =3

# Read in the data
df = pd.read_csv(f'humans-output-new-wiki-{which_wiki}.csv')

In [2]:
# create a id list for each row of df
df['id'] = df.index

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [3]:
# random switch human 1 and human 2 and their writings
for index, row in df.iterrows():
    if random.random() > 0.5:
        df.loc[index, 'Human 1'] = row['Human 2']
        df.loc[index, 'Human 2'] = row['Human 1']
        df.loc[index, 'Human 1 Writing'] = row['Human 2 Writing']
        df.loc[index, 'Human 2 Writing'] = row['Human 1 Writing']

In [4]:
import json
with open(f'../gpt-3/gpt-outputs/zero-shot-batch-{which_wiki}.json') as f:
    gpt3_zeroshot = json.load(f)

# few-shot
with open(f'../gpt-3/gpt-outputs/few-shot-batch-{which_wiki}.json') as f:
    gpt3_fewshot = json.load(f)

with open(f"muss-output-new-wiki-{which_wiki}.json") as f:
    muss = json.load(f)

# for new-wiki-3 t5-3B and 11B are txt
with open(f'T5-3B-new-wiki-3-output.txt') as f:
    t5_3B = f.readlines()
    for i in range(len(t5_3B)):
        t5_3B[i] = t5_3B[i].strip()

with open(f'T5-11B-new-wiki-3-output.txt') as f:
    t5_11B = f.readlines()
    for i in range(len(t5_11B)):
        t5_11B[i] = t5_11B[i].strip()

In [5]:
df['GPT-3-zero-shot'] = gpt3_zeroshot
df['GPT-3-few-shot'] = gpt3_fewshot
df['Muss'] = muss
df['T5-3B'] = t5_3B
df['T5-11B'] = t5_11B

In [6]:
# remove \n in Human 1 Writing and Human 2 Writing
df['Human 1 Writing'] = df['Human 1 Writing'].replace("\n", "", regex=True).apply(str)
df['Human 2 Writing'] = df['Human 2 Writing'].replace("\n", "", regex=True).apply(str)

# replace "  " with " "
df['Human 1 Writing'] = df['Human 1 Writing'].replace("  ", " ", regex=True).apply(str)
df['Human 2 Writing'] = df['Human 2 Writing'].replace("  ", " ", regex=True).apply(str)

In [7]:
df.to_csv(f'all-outputs-new-wiki-{which_wiki}-before-splitting.csv', index=False)

In [8]:
from string import punctuation
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

def remove_tokenization_artifacts(s, src):
    stokens = s.split()
    snew = s
    for i, token in enumerate(stokens):
        if i > 0 and i < len(stokens) - 1 and token in punctuation:
            substrboth = stokens[i - 1] + token + stokens[i + 1]
            substrleft = stokens[i - 1] + token
            substright = token + stokens[i + 1]
            if substrboth in src:
                snew = snew.replace(stokens[i - 1] + " " + token + " " + stokens[i + 1], substrboth)
            elif substrleft in src:
                snew = snew.replace(stokens[i - 1] + " " + token, substrleft)
            elif substright in src:
                snew = snew.replace(token + " " + stokens[i + 1], substright)

    snew = snew.replace("''", '"')
    snew = snew.replace(" .", ".")
    snew_rest = "" if len(snew) == 1 else snew[1:]
    if len(snew) > 0:
        snew = snew[0].capitalize() + snew_rest
    snew = snew.replace("-lrb-", "(").replace("-rrb-", ")")
    snew = snew.replace("-LRB-", "(").replace("-RRB-", ")")
    return snew

def reformat_output(output, original):
    output = remove_tokenization_artifacts(output, original)
    output = " || ".join(sent_tokenize(output))
    return output

[nltk_data] Downloading package punkt to /Users/yaod_1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
# reformat output with reformat_output function
for index, row in df.iterrows():
    df.loc[index, 'Human 1 Writing'] = reformat_output(row['Human 1 Writing'], row['Original Sentence'])
    df.loc[index, 'Human 2 Writing'] = reformat_output(row['Human 2 Writing'], row['Original Sentence'])
    df.loc[index, 'GPT-3-zero-shot'] = reformat_output(row['GPT-3-zero-shot'], row['Original Sentence'])
    df.loc[index, 'GPT-3-few-shot'] = reformat_output(row['GPT-3-few-shot'], row['Original Sentence'])
    df.loc[index, 'Muss'] = reformat_output(row['Muss'], row['Original Sentence'])
    df.loc[index, 'T5-3B'] = reformat_output(row['T5-3B'], row['Original Sentence'])
    df.loc[index, 'T5-11B'] = reformat_output(row['T5-11B'], row['Original Sentence'])

In [10]:
df.to_csv(f'all-outputs-new-wiki-{which_wiki}.csv', index=False)